In [2]:
import pandas as pd

df = pd.read_csv('../data/processed/cleaned_salary_data.csv')
df.head()


,unnamed:_0,rating,founded,hourly,employer_provided,min_salary,max_salary,avg_salary,same_state,age,...,job_state_WA,job_state_WI,job_simp_data engineer,job_simp_data scientist,job_simp_director,job_simp_manager,job_simp_mle,job_simp_na,seniority_na,seniority_senior
0,0,3.8,1973,0,0,53,91,72.0,0,47,...,False,False,False,True,False,False,False,False,True,False
1,1,3.4,1984,0,0,63,112,87.5,0,36,...,False,False,False,True,False,False,False,False,True,False
2,2,4.8,2010,0,0,80,90,85.0,1,10,...,False,False,False,True,False,False,False,False,True,False
3,3,3.8,1965,0,0,56,97,76.5,1,55,...,True,False,False,True,False,False,False,False,True,False
4,4,2.9,1998,0,0,86,143,114.5,1,22,...,False,False,False,True,False,False,False,False,True,False


In [3]:
# Split data into Features X and Targets y
X = df.drop('avg_salary', axis=1)
y = df['avg_salary']


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
#Train Baseline model - Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


R²: 0.9995010232384852
MAE: 0.3436607112686374


In [6]:
#Try stronger model - Non-linear Regression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print("RF R²:", r2_score(y_test, rf_pred))
print("RF MAE:", mean_absolute_error(y_test, rf_pred))


RF R²: 0.9982828860652634
RF MAE: 0.7845637583892624


In [7]:
#Can also try XGBoost
from xgboost import XGBRegressor
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/chenxuanzhang/Documents/my-AI-projects/SalaryPrediction/venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B111F8D5-6AC6-3245-A6B5-94693F6992AB> /Users/chenxuanzhang/Documents/my-AI-projects/SalaryPrediction/venv/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/

6️⃣ Compare model performance
Model	R²	MAE
Linear Regression	…	…
Random Forest	…	…
Gradient Boosting	…	…

Pick the best-performing one as your “production” model.

7️⃣ Save the best model
Once you’re happy with the results:

In [ ]:
import joblib
joblib.dump(rf, 'app/model.pkl')


8️⃣ (Optional) Feature importance

To see which features matter most:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title("Feature Importances")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.tight_layout()
plt.show()
